<div style="text-align: center;"> 
    <strong>
        Setup
    <strong>
</div>

In [197]:
import pandas as pd
import os
import pyreadstat

path = os.path.join("..","Interview_Spec","Test Data.sav")
outputCSVPath = os.path.join("..","Data","Test_Data.csv")
# Os.path.join makes the path operating system independent


df, meta = pyreadstat.read_sav(path)
#print(df.head(1))

print(meta.column_names)  # List of column names
print(meta.column_labels)  # List of column labels
print(meta.notes)
print(meta.creation_time)
# No notes on the dataSet so assumptions need to be made based on the data.

['respid', 'q01', 'q02', 'q03_uk', 'q03_us', 'country', 'q04_uk', 'q04_us', 'q06_uk', 'q06_us', 'q07', 'q09_01', 'q09_02', 'q09_03', 'q09_04', 'q09_05', 'q09_06', 'q09_07', 'q09_08', 'q09_09', 'q09_10', 'q09_11', 'q09_12', 'q09_13', 'q09_14', 'q09_15', 'q09_16', 'q09_17', 'q09_18', 'q09_19', 'q09_20', 'q09_21', 'q09_22', 'q09_23', 'q09_24', 'q09_25', 'q09_26', 'q09_27', 'q09_28', 'q09_29', 'q09_30', 'q09_31', 'q09_32', 'q09_33', 'q09_34', 'q10_01', 'q10_02', 'q10_03', 'q10_04', 'q10_05', 'q10_06', 'q10_07', 'q10_08', 'q10_09', 'q10_10', 'q10_12', 'q10_13', 'q10_15', 'q10_18', 'q10_19', 'q10_20', 'q10_21', 'q10_23', 'q10_26', 'q10_28', 'q10_30', 'q10_31', 'q10_32', 'q11_01', 'q11_02', 'q11_03', 'q11_04', 'q11_05', 'q11_06', 'q11_07', 'q11_08', 'q11_09', 'q11_10', 'q11_12', 'q11_13', 'q11_15', 'q11_18', 'q11_19', 'q11_20', 'q11_21', 'q11_23', 'q11_26', 'q11_28', 'q11_30', 'q11_31', 'q11_32', 'q12_01', 'q12_02', 'q12_03', 'q12_04', 'q12_05', 'q12_06', 'q12_07', 'q12_08', 'q12_09', 'q12_10

<div style="text-align: center;"> 
<strong>
Helper Functions
<strong>
</div>

In [198]:
#HelperFunctions
def questionToVariableName(question:str)->str:
    # returns the variable Name from a column label: 
    # ('Q.1 AGE')->(q01)
    questionsToColNames = dict(zip(meta.column_labels,meta.column_names)) # not efficient but good enough for purpose
    return questionsToColNames.get(question)
def getQuestionKey(question:str)->dict[str,list[str]]:
    # returns the encoding for a specific question:
    # (IS Bobbi Brown good value for money)-> {1:Strongly agree, 2:agree ..etc}
    # This will only return the first mapping of a question when there are repeated questions
    return meta.variable_value_labels.get(questionToVariableName(question))
def getQuestionKeyWithColumnName(column:str)->dict[str,list[str]]:
    return meta.variable_value_labels.get(column)

<div style="text-align: center;"> 
<strong>
The Data
<strong>
<br>

</div>

In [199]:
#How many brands are in the survey?
import re
brands = [re.search("Q.16 Offers good value for money: (.*)|",column).group(1) \
                  for column in meta.column_labels if "value" in column.lower()]
print(f"Brands surveyed: {brands}")
print(f"Number of brands: {len(brands)}")
#How many participants?
print(f"Number of people surveyed {len(df)}")



Brands surveyed: ['Charlotte Tilbury', 'Benefit Cosmetics', 'Bobbi Brown', 'La Mer', 'EstÚe Lauder', 'Yves Saint Laurent Beauty', 'MAC', 'Dior Beauty', 'NARS', 'Tom Ford Beauty', 'Anastasia Beverly Hills', 'Chanel', 'Huda Beauty', 'Too Faced', 'Pat McGrath', 'Glossier', 'Fenty Beauty', 'IT Cosmetics', 'Clarins', 'Laura Mercier', 'Clinique', 'Rare Beauty', 'ELF']
Number of brands: 23
Number of people surveyed 1233


<div style="text-align: center;"> 
<strong>
Recipient Profile
<strong>
<br>

</div>

In [200]:
# Male/female split
genderColumn= questionToVariableName("Q.2 Gender")
print(getQuestionKey("Q.2 Gender"))
maleSubset = df[df[genderColumn]==1]
femaleSubset = df[df[genderColumn]==2]
otherSubset = df[df[genderColumn]==3]
print(f"Number of male recipients: {len(maleSubset)}")
print(f"Number of female recipients: {len(femaleSubset)}")
# Age Split
print()
ageColumn = questionToVariableName("Q.1 Age")
print(getQuestionKey("Q.1 Age"))
ageSubsets = {ageRange: df[df[ageColumn]==key] for key,ageRange in getQuestionKey("Q.1 Age").items()}
for ageRange,subset in ageSubsets.items():
    print(f"Number of participants {ageRange}: {len(subset)}")

ukSubset= df[df['country']==1]
usSubset = df[df['country']==2]
print()
#Country split
print(f"Number of participants in the UK :{len(ukSubset)}")
majorUKCitySubsets = {city:df[df['q04_uk']==key] for key,city in getQuestionKeyWithColumnName('q04_uk').items()}
for city,subset in majorUKCitySubsets.items():
    print(f"number of participants living near in {city} : {len(subset)}")
print()
print(f"Number of participants in the US :{len(usSubset)}")
majorUSCitySubsets = {city:df[df['q04_us']==key]for key,city in getQuestionKeyWithColumnName('q04_us').items()}


{1.0: 'Male', 2.0: 'Female', 3.0: 'Other', 4.0: 'Prefer not to say'}
Number of male recipients: 0
Number of female recipients: 1233

{1.0: 'Under 16', 2.0: '16 to 24', 3.0: '25 to 34', 4.0: '35 to 44', 5.0: '45 to 54', 6.0: '55 to 64', 7.0: '65 or older', 8.0: 'Prefer not to say'}
Number of participants Under 16: 0
Number of participants 16 to 24: 306
Number of participants 25 to 34: 231
Number of participants 35 to 44: 218
Number of participants 45 to 54: 223
Number of participants 55 to 64: 255
Number of participants 65 or older: 0
Number of participants Prefer not to say: 0

Number of participants in the UK :500
number of participants living near in Birmingham : 35
number of participants living near in London : 74
number of participants living near in Leeds : 22
number of participants living near in Liverpool : 21
number of participants living near in Manchester : 20
number of participants living near in None of these : 328

Number of participants in the US :733


<div style="text-align: center;"> 
<strong>
Recipient Profile 2
<strong>
<br>

</div>

In [201]:
# Income split
import csv
ukIncomeSubsets ={incomeBracket.replace('ú','£'):df[df['q06_uk']==key] for key,incomeBracket in getQuestionKeyWithColumnName('q06_uk').items()}
for income, subset in ukIncomeSubsets.items():
    print(f"Number of UK particpants earning {income} : {len(subset)}")


print("")
usIncomeSubsets ={incomeBracket:df[df['q06_us']==key] for key,incomeBracket in getQuestionKeyWithColumnName('q06_us').items()}
for income, subset in usIncomeSubsets.items():
    print(f"Number of US particpants earning {income} : {len(subset)}")



Number of UK particpants earning Up to £7,000 : 0
Number of UK particpants earning £7,001 to £14,000 : 0
Number of UK particpants earning £14,001 to £21,000 : 0
Number of UK particpants earning £21,001 to £28,000 : 0
Number of UK particpants earning £28,001 to £34,000 : 70
Number of UK particpants earning £34,001 to £41,000 : 71
Number of UK particpants earning £41,001 to £48,000 : 58
Number of UK particpants earning £48,001 to £55,000 : 78
Number of UK particpants earning £55,001 to £62,000 : 59
Number of UK particpants earning £62,001 to £69,000 : 38
Number of UK particpants earning £69,001 to £76,000 : 31
Number of UK particpants earning £76,001 to £83,000 : 28
Number of UK particpants earning £83,001 or more : 67
Number of UK particpants earning Prefer not to say : 0

Number of US particpants earning Up to $14,000 : 0
Number of US particpants earning $14,001 to $25,000 : 0
Number of US particpants earning $25,001 to $35,000 : 0
Number of US particpants earning $35,001 to $49,999 : 

<div style="text-align: center;"> 
<strong>
saving recipient data
<strong>
<br>

</div>

In [202]:
ukIncomeCSVFileName = os.path.join("..","Data","UKIncomeBrackets.csv")
try:
    with open(ukIncomeCSVFileName,mode="w",newline="") as file:
        if os.path.exists(ukIncomeCSVFileName):
            raise FileExistsError(f"The file '{ukIncomeCSVFileName}' already exists.")
        writer = csv.writer(file)
        writer.writerow(["Income","NO."])
        for incomeBracket,subset in ukIncomeSubsets.items():
            writer.writerow([incomeBracket,len(subset)])
except Exception as fee:
    print(fee)
except Exception as e:
    print(e)

usIncomeCSVFileName = os.path.join("..","Data","USIncomeBrackets.csv")
try:
    with open(usIncomeCSVFileName,mode="w",newline="") as file:
        if os.path.exists(usIncomeCSVFileName):
            raise FileExistsError(f"The file '{usIncomeCSVFileName}' already exists.")
        writer = csv.writer(file)
        writer.writerow(["Income","NO."])
        for incomeBracket,subset in usIncomeSubsets.items():
            writer.writerow([incomeBracket,len(subset)])
except Exception as fee:
    print(fee)
except Exception as e:
    print(e)    


The file '..\Data\UKIncomeBrackets.csv' already exists.
The file '..\Data\USIncomeBrackets.csv' already exists.


<div style="text-align: center;"> 
<strong>
PROCESSING:
<strong>
<br>
How many Respondents in the UK think Bobbi Brown offers good value for money?
</div>

In [203]:

bobbiColumnMentions = [column for column in meta.column_labels if "bobbi brown" in column.lower()]
print(bobbiColumnMentions)
#there are 49 columns that mention "Bobbi Brown" to choose from and luckily the corect one includes "Value"
bobbiValueQuestion = [column for column in bobbiColumnMentions if "value" in column.lower()][0]
bobbiQuestionVariableName = questionToVariableName(bobbiValueQuestion)
countryVariableName = questionToVariableName('Country')

print(getQuestionKey(bobbiValueQuestion))

# values 1.0 and 2.0 correlate to agreeeing that Bobbi Brown is good value for money
print(getQuestionKey('Country'))
filteredDF= df[(df['country']==1.0) & (df[bobbiQuestionVariableName]<=2.0)]
print(len(filteredDF))


['Q.9 Which of the following make-up, skincare, and/or fragrance brands have you heard of? Prompted Awareness - Bobbi Brown', 'Q.10 How much, if anything, would you say you know about each of the following brands? Bobbi Brown', 'Q.11 Have you purchased or browsed make-up, skincare and/or fragrance products from the following brands? Bobbi Brown', 'Q.12 How often do you purchase make-up, skincare and/or fragrance products from the following brands? Bobbi Brown', 'Q.12b Do you think the following brands are growing or declining in popularity? Bobbi Brown', 'Q.13 Which of the following brands would you consider purchasing beauty or skincare products from in the future? Bobbi Brown', 'Q.15 How likely are you to recommend the following brands to a friend or family members? Bobbi Brown', 'Q.16 Easy to use: Bobbi Brown', 'Q.16 Easy to select the product I need: Bobbi Brown', 'Q.16 Easy to select the right gift: Bobbi Brown', "Q.16 Created by the World's ^1 Make-Up Artist: Bobbi Brown", 'Q.16 

<div style="text-align: center;"> 
<strong>
Further Ideas
<strong>
</div>
<br>
Difference between UK and US
<br>
Test for difference between regions and potentially compare with average income of the region
<br>
compare with different products
<br>
check the average cost for a particular product shared between items and then model it with the data
<br>
normalise each respondents value for money based on their veiws over other products
<br>
luxury brands sell more in afluent major cities. Would be useful to categorise respondents in major cities and give them priority queries as their opinions may be more important than others. 
<br>
check if it possible to categorise certain respondents based on their overall "Value for money" results.
<br>
categorise low,mid and high income respondents 
<br>
categorise respondents based on whether they buy make-up skin care etc. 
<br>
find the metric used for the weighting column and give priority based on respondents with higher weight


<div style="text-align: center;"> 
<strong>
PROCESSING:
<strong>
<br>
Out of all respondents, how many agree that at least 3 brands embrace diversity?
</div>

In [204]:
diversityColumnMentions = [column for column in meta.column_labels if "diversity" in column.lower()]
print(diversityColumnMentions)
print(len(diversityColumnMentions))
diversityColumnVariables = [questionToVariableName(column) for column in diversityColumnMentions]
dfDiversityQuestionSubset = df[diversityColumnVariables]
dfDiversityAgreementNO = (dfDiversityQuestionSubset<=2.0).sum(axis='columns')  # number of diversity satisfactions per person
diversityConditionSatisfied = (dfDiversityAgreementNO>=3).sum()  # number of people that agree to atleast 3 brand diversity questions
print(diversityConditionSatisfied)


['Q.16 Embraces diversity: Charlotte Tilbury', 'Q.16 Embraces diversity: Benefit Cosmetics', 'Q.16 Embraces diversity: Bobbi Brown', 'Q.16 Embraces diversity: La Mer', 'Q.16 Embraces diversity: EstÚe Lauder', 'Q.16 Embraces diversity: Yves Saint Laurent Beauty', 'Q.16 Embraces diversity: MAC', 'Q.16 Embraces diversity: Dior Beauty', 'Q.16 Embraces diversity: NARS', 'Q.16 Embraces diversity: Tom Ford Beauty', 'Q.16 Embraces diversity: Anastasia Beverly Hills', 'Q.16 Embraces diversity: Chanel', 'Q.16 Embraces diversity: Huda Beauty', 'Q.16 Embraces diversity: Too Faced', 'Q.16 Embraces diversity: Pat McGrath', 'Q.16 Embraces diversity: Glossier', 'Q.16 Embraces diversity: Fenty Beauty', 'Q.16 Embraces diversity: IT Cosmetics', 'Q.16 Embraces diversity: Clarins', 'Q.16 Embraces diversity: Laura Mercier', 'Q.16 Embraces diversity: Clinique', 'Q.16 Embraces diversity: Rare Beauty', 'Q.16 Embraces diversity: ELF']
23
334


<div style="text-align: center;"> 
<strong>
PROCESSING:
<strong>
<br>
What factors determine whether someone is likely to reccomend a brand to someone else?
</div>

In [205]:
#Modelling Functions

#Cleaning Data
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

def reCodeAndClean(encodingMapping:dict[any,any],df=df,naValue=0,replaceNAValues = True):
    cleanedDF = df.replace(encodingMapping)
    return cleanedDF.fillna(naValue) if replaceNAValues else cleanedDF
def runlinearModel(target,independentVariables=df):
    X_train, X_test, y_train, y_test = train_test_split(independentVariables,target, test_size=0.2, random_state=42)
    linearModel = LinearRegression()
    linearModel.fit(X_train,y_train)


    y_pred = linearModel.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print("Linear Model")
    print("Mean Squared Error (MSE):", mse)
    print("R² Score:", r2)
    print("")
    return r2
    


def runLassoModel(target,independentVariables=df):
    X_train, X_test, y_train, y_test = train_test_split(independentVariables,target, test_size=0.2, random_state=42)
    linearModel = Lasso()
    linearModel.fit(X_train,y_train)
    # Initialize Lasso with a regularization parameter (alpha)
    X_train, X_test, y_train, y_test = train_test_split(independentVariables, target, test_size=0.2, random_state=42)
    lasso = Lasso(alpha=0.1)
    # Fit the model
    lasso.fit(X_train, y_train)
    # Get coefficients
    print("Lasso Model")
    print("Coefficients:", lasso.coef_)
    # Predict on the test set
    y_pred = lasso.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print("Mean Squared Error:", mse)
    print("R^2 Score:", r2)
    print("")
    return [float(value)for value in lasso.coef_]




<div style="text-align: center;"> 
<strong>
PROCESSING:
<strong>
<br>
Predicting reccomendations for Charlotte
</div>

In [206]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


q16Predictors = [questionToVariableName(column) for column in meta.column_labels \
                           if "q.16" in column.lower()\
                            and "charlotte" in column.lower()]
q16Questions = [column for column in meta.column_labels \
                           if "q.16" in column.lower()\
                            and "charlotte" in column.lower()]

q15Target = [questionToVariableName(column) for column in meta.column_labels\
         if "q.15" in column.lower() and\
            "charlotte" in column.lower()]

CleanedMapping = {1.0:2.0,2.0:1.0,3.0:0.0,4.0:-1.0,5.0:-2.0,6.0:0.0}
cleanedq16Predictors = reCodeAndClean(CleanedMapping,naValue=0,df=df[q16Predictors])#df[q16Predictors].replace(CleanedMapping).fillna(0)
cleanedq15Target = df[q15Target].fillna(5.0)

runlinearModel(cleanedq15Target,cleanedq16Predictors)
print("")
coeffiencts = runLassoModel(cleanedq15Target,cleanedq16Predictors)
# less than 50% of the variance in reccomendation can be explained by the survey using this model
questionsRelevance = dict(zip(q16Questions,coeffiencts))
print(questionsRelevance)
#best predictors in order: is a brand I feel proud to support 0.57
#easy to select the product I need 0.41
#Offers best Quality Product
#iconic 0.31
#
# Note in presentation that onl simple models were used because the data is simple and fairly sparse. 
# Using complex models like NN increase the chance for overfitting
#

Linear Model


Mean Squared Error (MSE): 2.45780823951255
R² Score: 0.47962401047904535


Lasso Model
Coefficients: [ 0.15719855  0.4153245   0.19268646  0.          0.          0.
  0.          0.          0.3139061   0.02182392  0.34955712  0.
  0.          0.          0.1202511   0.57245069  0.          0.
  0.          0.         -0.        ]
Mean Squared Error: 2.4264438105067603
R^2 Score: 0.4862646000568899

{'Q.16 Easy to use: Charlotte Tilbury': 0.1571985515949273, 'Q.16 Easy to select the product I need: Charlotte Tilbury': 0.4153244993065185, 'Q.16 Easy to select the right gift: Charlotte Tilbury': 0.19268645773346132, "Q.16 Created by the World's ^1 Make-Up Artist: Charlotte Tilbury": 0.0, 'Q.16 Used by Celebrities: Charlotte Tilbury': 0.0, 'Q.16 Teaches me about red carpet beauty: Charlotte Tilbury': 0.0, 'Q.16 Offers personalised make-up and beauty advice: Charlotte Tilbury': 0.0, 'Q.16 Luxury: Charlotte Tilbury': 0.0, 'Q.16 Iconic: Charlotte Tilbury': 0.31390610224403637, 'Q.16 Empowe

<div style="text-align: center;"> 
<strong>
PROCESSING:
<strong>
<br>
Predicting Reccomendations: generalised
</div>

In [207]:
# Predicting rec
def brandReccomendations(brandName,df=df):
    q16Predictors = [questionToVariableName(column) for column in meta.column_labels \
                           if "q.16" in column.lower()\
                            and brandName in column.lower()]
    q16Questions = [column for column in meta.column_labels \
                            if "q.16" in column.lower()\
                                and brandName in column.lower()]

    q15Target = [questionToVariableName(column) for column in meta.column_labels\
            if "q.15" in column.lower() and\
                brandName in column.lower()]

    CleanedMapping = {1.0:2.0,2.0:1.0,3.0:0.0,4.0:-1.0,5.0:-2.0,6.0:0.0}
    cleanedq16Predictors = reCodeAndClean(CleanedMapping,naValue=0,df=df[q16Predictors])#df[q16Predictors].replace(CleanedMapping).fillna(0)
    cleanedq15Target = df[q15Target].fillna(5.0)

    runlinearModel(cleanedq15Target,cleanedq16Predictors)
    print("")
    coeffiencts = runLassoModel(cleanedq15Target,cleanedq16Predictors)
    # less than 50% of the variance in reccomendation can be explained by the survey using this model
    questionsRelevance = dict(zip(q16Questions,coeffiencts))
    print(sorted(questionsRelevance.items(),key= lambda item: item[1],reverse=True))



In [214]:
brandReccomendations("bobbi",)

Linear Model
Mean Squared Error (MSE): 4.650628162566088
R² Score: 0.0812025233642012


Lasso Model
Coefficients: [0.         0.         0.         0.06356375 0.         0.
 0.         0.         0.         0.         0.01384998 0.
 0.         0.         0.45455227 0.23294848 0.         0.
 0.         0.        ]
Mean Squared Error: 4.917977238478922
R^2 Score: 0.02838392601710915

[('Q.16 Offers good value for money: Bobbi Brown', 0.4545522707735025), ('Q.16 Is a brand I feel proud to support: Bobbi Brown', 0.23294847706006339), ("Q.16 Created by the World's ^1 Make-Up Artist: Bobbi Brown", 0.06356375207460441), ('Q.16 Offers the best quality products: Bobbi Brown', 0.013849980400824358), ('Q.16 Easy to use: Bobbi Brown', 0.0), ('Q.16 Easy to select the product I need: Bobbi Brown', 0.0), ('Q.16 Easy to select the right gift: Bobbi Brown', 0.0), ('Q.16 Used by Celebrities: Bobbi Brown', 0.0), ('Q.16 Teaches me about red carpet beauty: Bobbi Brown', 0.0), ('Q.16 Offers personalised mak

In [209]:
brandReccomendations("dior")

Linear Model
Mean Squared Error (MSE): 6.862298208613276
R² Score: 0.05128564764241683


Lasso Model
Coefficients: [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.10567965 0.11774506 0.
 0.         0.         0.78421015 0.30933198 0.         0.
 0.         0.         0.        ]
Mean Squared Error: 6.838443950446829
R^2 Score: 0.05458350448270188

[('Q.16 Offers good value for money: Dior Beauty', 0.784210149382431), ('Q.16 Is a brand I feel proud to support: Dior Beauty', 0.30933197635046344), ('Q.16 Offers the best quality products: Dior Beauty', 0.11774506045639546), ('Q.16 Empowers women to be the best version of themselves: Dior Beauty', 0.10567964890023855), ('Q.16 Easy to use: Dior Beauty', 0.0), ('Q.16 Easy to select the product I need: Dior Beauty', 0.0), ('Q.16 Easy to select the right gift: Dior Beauty', 0.0), ("Q.16 Created by the World's ^1 Make-Up Artist: Dior Beauty", 0.0), ('Q.16 Used by Celebrities: Dior Beauty', 0.0), ('Q

In [213]:
brandReccomendations("la mer")

Linear Model
Mean Squared Error (MSE): 1.194970652629034
R² Score: 0.16977218892355572


Lasso Model
Coefficients: [0.         0.         0.         0.         0.         0.00752975
 0.         0.         0.         0.02989832 0.         0.
 0.         0.         0.48602094 0.09600022 0.11822668 0.
 0.         0.        ]
Mean Squared Error: 1.2489386532118822
R^2 Score: 0.13227692919186762

[('Q.16 Offers good value for money: La Mer', 0.486020936549617), ('Q.16 Uses scientifically proven products/ingredients: La Mer', 0.11822668424392375), ('Q.16 Is a brand I feel proud to support: La Mer', 0.09600022172623478), ('Q.16 Empowers women to be the best version of themselves: La Mer', 0.02989831943233029), ('Q.16 Teaches me about red carpet beauty: La Mer', 0.007529753451599013), ('Q.16 Easy to use: La Mer', 0.0), ('Q.16 Easy to select the product I need: La Mer', 0.0), ('Q.16 Easy to select the right gift: La Mer', 0.0), ("Q.16 Created by the World's ^1 Make-Up Artist: La Mer", 0.0), ('Q

<div style="text-align: center;"> 
<strong>
PROCESSING:
<strong>
<br>
Optimising the impact of Strongly agree/disagree to agree/disagree
</div>

In [ ]:
from scipy.optimize import minimize

initialGuess = 2 # double the value of agree
